## ДЗ по поиску

Привет! Вам надо реализивать поисковик на базе вопросов-ответов с сайта [pravoved.ru](https://pravoved.ru/questions-archive/).        
Поиск должен работать на трех технологиях:       
1. обратном индексе     
2. word2vec         
3. doc2vec      

Вы должны понять, какой метод и при каких условиях эксперимента на этом корпусе работает лучше.          
Для измерения качества поиска найдите точность (accuracy) выпадания правильного ответа на конкретный вопрос (в этой базе у каждого вопроса есть только один правильный ответ). Точность нужно измерить для всей базы.    
При этом давайте считать, что выпал правильный ответ, если он попал в **топ-5** поисковой выдачи.

> Сделайте ваш поиск максимально качественным, чтобы значение точности стремилось к 1.     
Для этого можно поэкспериментировать со следующим:       
- модель word2vec (можно брать любую из опен сорса или обучить свою)
- способ получения вектора документа через word2vec: простое среднее арифметическое или взвешивать каждый вектор в соответствии с его tf-idf      
- количество эпох у doc2vec (начинайте от 100)
- предобработка документов для обучения doc2vec (удалять / не удалять стоп-слова)
- блендинг методов поиска: соединить результаты обратного индекса и w2v, или (что проще) w2v и d2v

На это задание отведем 10 дней. Дэдлайн сдачи до полуночи 12.10.

In [156]:
import pickle

with open('qa_corpus.pkl', 'rb') as file:
    qa_corpus = pickle.load(file)

Всего в корпусе 1384 пары вопрос-ответ

In [157]:
len(qa_corpus)

1384

Первый элемент блока это вопрос, второй - ответ на него

In [158]:
qa_corpus[0]

['\nДобрый день.Мой сын гражданин Украины (ДНР),имеет вид на жительство в Р.Ф., кот.получил проживая с 2014 г. в Нижегородской области.В 2017г. переехал на постоянное место жительство в г.Ростов.Официально трудоустроился на одно из промышл.предприятий г.Ростова.Оформил временную регистрацию в Ростове.В УФМС предупредили,что по истечении 90 дней он должен либо постоянно прописаться либо покинуть территорию России.Прошу проконсультировать как быть дальше.(Вернуться домой в Донецк,но здесь идет война,работы нет.В Ростове он работает по специальности.Он инженер машиностроитель.)Временная прописка до 15 марта.  Если он сможет приобрести какую либо недвижимость,как долго будет решаться вопрос о его постоянной прописке в Ростове.Как в этом случае будет решаться вопрос с видом на жительство в Ростове? Не получится ли ,что приобретя квартиру,он не успеет в ней прописаться до окончании срока временной регистрации. С уважением Людмила Евгеньевна.\n',
 'Добрый вечер!Из Вашего вопроса вообще ничего

In [159]:
import gensim
import string
import re
import json
from pymystem3 import Mystem
from gensim import matutils
import numpy as np 
import pandas as pd
from nltk import word_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec, KeyedVectors
from collections import defaultdict
import warnings

warnings.filterwarnings('ignore')
mystem = Mystem()

In [160]:
model_path = '/Users/macbook/Downloads/araneum_none_fasttextcbow_300_5_2018/araneum_none_fasttextcbow_300_5_2018.model'
model = Word2Vec.load(model_path)

In [161]:
def preprocessing(input_text, del_stopwords=True, del_digit=True):
    """
    :input: raw text
        1. lowercase, del punctuation, tokenize
        2. normal form
        3. del stopwords
        4. del digits
    :return: lemmas
    """
    russian_stopwords = set(stopwords.words('russian'))
    words = [x.lower().strip(string.punctuation+'»«–…') for x in word_tokenize(input_text)]
    lemmas = [mystem.lemmatize(x)[0] for x in words if x]

    lemmas_arr = []
    for lemma in lemmas:
        if del_stopwords:
            if lemma in russian_stopwords:
                continue
        if del_digit:
            if lemma.isdigit():
                continue
        lemmas_arr.append(lemma)
    return lemmas_arr

In [162]:
done = pred_data

In [163]:
def w2v_vector(word_list):
    some = []
    for word in word_list:
        try:
            some.append(model.wv[str(word)])
        except KeyError:
            continue
    return matutils.unitvec(np.array(sum(some)))

In [102]:
index = 1

data = defaultdict()

for item in qa_corpus:
    data[index] = [w2v_vector(preprocessing(item[0])), item[1]]
    index += 1

In [113]:
def search(query):
    query_vector = w2v_vector(preprocessing(query))
    local_data = defaultdict()
    for item in data:
        local_data[similarity(query_vector, data[item][0])] = item 
    for el in sorted(local_data, reverse=True)[:5]:
        yield local_data[el]

In [109]:
def similarity(v1, v2):
    return np.dot(v1, v2)

In [121]:
list(search('Как жить в России'))

[1352, 947, 817, 843, 625, 793, 791, 798, 1071, 703]

In [146]:
def search(query):
    query_vector = w2v_vector(preprocessing(query))
    local_data = defaultdict()
    for item in data:
        local_data[similarity(query_vector, data[item][0])] = item 
    for el in sorted(local_data, reverse=True)[:5]:
        yield data[local_data[el]][1]

In [141]:
def testing():
    print(data[1])

In [142]:
testing()

[array([  7.15986267e-02,   4.11028452e-02,  -8.47221687e-02,
         4.86208946e-02,  -1.26518831e-01,  -3.15667130e-02,
         9.50335190e-02,   1.51726818e-02,   1.07730515e-01,
         3.60690504e-02,  -6.45365007e-03,  -6.23325519e-02,
        -2.51616500e-02,   1.95927210e-02,   1.80515628e-02,
        -8.75344649e-02,   8.88505057e-02,   6.65214669e-04,
        -2.43865643e-02,  -4.21775728e-02,   4.49636392e-02,
        -1.56622469e-01,   2.01686174e-02,   5.88442999e-05,
         3.48556833e-03,  -6.02354528e-03,  -3.61343436e-02,
         1.87353920e-02,  -3.14579234e-02,  -5.91415800e-02,
        -8.81366283e-02,  -8.79356638e-02,   1.28511153e-02,
         4.13270928e-02,   1.14664324e-01,   7.63024390e-02,
         5.22525422e-03,  -1.04037803e-02,  -4.73733526e-03,
        -1.71672292e-02,   6.65300526e-03,   6.07886985e-02,
        -5.15172891e-02,   7.87028745e-02,   2.72175595e-02,
        -5.55477068e-02,   4.33116071e-02,   3.12198531e-02,
         7.64377639e-02

In [171]:
test_data = defaultdict()

true = 0

for qa in qa_corpus:
    if qa[1] in list(search(qa[0])):
        true += 1

print('For {} attempts, there are(is) {} correct matches (score:{})'.format(len(qa_corpus), true, true / len(qa_corpus)))

For 1384 attempts, there are(is) 1367 correct matches (score:0.9877167630057804)


In [168]:
true, true / len(qa_corpus)

(1367, 0.9877167630057804)